In [14]:
%matplotlib inline
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

from basegame.game import Game
from basegame.player import Player

In [15]:
from basegame.nazwisko import Nazwisko
from benchmarks.simplePlayer import SimplePlayer
from benchmarks.simpleNoCheck import SimpleNoCheck
from benchmarks.simpleAlwaysCheck import SimpleAlwaysCheck
from absolutnie_do_wywalenia.drawPlayer import DrawPlayer
from benchmarks.honestPlayer import HonestPlayer

from moje_dzieci.MyFirstPlayer import MyFirstPlayer
from moje_dzieci.MySecondPlayer import MySecondPlayer
from moje_dzieci.MyThirdPlayer import MyThirdPlayer
from absolutnie_do_wywalenia.cardCounter import FirstCardCounter
from absolutnie_do_wywalenia.cardCounter_base2 import SecondCardCounter
from moje_dzieci.SecondOptim import MySecondPlayerOptim
from czyje.Kondys_Dabrowski import Kondys_Dabrowski
from ostatnie_pokolenie.Kotterek3 import Kotterek3
from czyje.Rafalek import Rafalek

In [16]:
### Generate cards from 9 to 14 (ace) for all colors/symbols (0, 1, 2, 3)
def getDeck():
    return [(number, color) for color in range(4) for number in range(9, 15)]
    
print(getDeck())

[(9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (9, 3), (10, 3), (11, 3), (12, 3), (13, 3), (14, 3)]


In [17]:
### Shuffle the cards randomly. Each player gets 9 cards
### (so one player cannot be certain which cards the other player has)

def getShuffled(deck):
    D = set(deck)
    A = set(random.sample(deck, 8))
    B = set(random.sample(list(D - A), 8))
    C = D - A - B
    if len(A.intersection(B)) > 0: print("Shuffle error 1")
    if len(A.intersection(B)) > 0: print("Shuffle error 2")
    if len(A.intersection(C)) > 0: print("Shuffle error 3") 
    DS = A | B | C
    if not DS == D: print("Shuffle error 4")  
    return list(A), list(B), list(C)

p1, p2, notUsed, = getShuffled(getDeck())
print(p1)
print(p2)


[(13, 1), (11, 3), (9, 2), (12, 3), (13, 3), (14, 3), (9, 1), (10, 2)]
[(11, 1), (9, 0), (12, 1), (11, 0), (14, 1), (12, 0), (13, 0), (10, 0)]


In [18]:
# Some examplary random player

class RandomPlayer(Player):
    
    ### player's random strategy
    def putCard(self, declared_card):
        
        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        
        ### player randomly decides which card put on the table
        card = random.choice(self.cards)
        declaration = card
        
        ### player randomly decides whether to cheat or not
        cheat = np.random.choice([True, False])
       
        ### if (s)he decides to cheat, (s)he randomly declares the card.
        if cheat:
            declaration = random.choice(self.cards)             
            
        ### Yet, declared card should be no worse than a card on the top of the pile . 
        if declared_card is not None and declaration[0] < declared_card[0]:
            declaration = (min(declared_card[0]+1,14), declaration[1])

        ### return the decision (true card) and declaration (player's declaration)
        return card, declaration
    
    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([True, False])
    

Analyze few moves...

In [19]:
### Perform a full game 100 times
STAT_NAMES = ["Wins", "Draws", "Moves", "Cards", "Pile Size", "Checks", "Draw Decisions", "Cheats", "Errors", "Total errors"]
ANALYZE_STATS = [0, 1, 2, 3, 5, 6, 7, 8]

def printResults(results):
    print("Wins:")
    print(results[0])
    print("Draws:")
    print(results[1])
    print("Moves:")
    print(results[2])
    print("Cards:")
    print(results[3])
    print("Pile size:")
    print(results[4])
    print("Checks:")
    print(results[5])
    print("Draw decisions:")
    print(results[6])
    print("Cheats:")
    print(results[7])
    print("Errors:")
    print(results[8])
    print("Total errors:")
    print(results[9])

def comparePlayers(player1_class, player2_class):
    stats_wins = [0, 0]
    stats_draws = [0, 0]
    stats_moves = [0, 0]
    stats_cards = [0, 0]
    stats_pile_size = 0
    stats_checks = [0, 0]
    stats_draw_decisions = [0, 0]
    stats_cheats = [0, 0]
    stats_errors = [0, 0]
    
    repeats = 1000
    errors = 0
    draws = 0

    for t in range(repeats):
        player1 = player1_class("")
        player2 = player2_class("")
        game = Game([player1, player2], log = False)
    
        error = False
        draw = False
        
        while True:
            valid, player = game.takeTurn(log = False)
            if game.moves[0] > 100 or game.moves[1] > 100:
                draws += 1
                stats_draws[0] += 1
                stats_draws[1] += 1
                if (game.player_cards[0] < game.player_cards[1]):
                    stats_wins[0] += 1
                if (game.player_cards[0] > game.player_cards[1]):
                    stats_wins[1] += 1
                    
                    
                draw=True
                # print("DRAW")
                break
            if not valid:
                error = True
                stats_errors[player] += 1
                errors += 1
                break
            if game.isFinished(log = False):
                stats_wins[player] += 1
                break
            
        stats_pile_size += len(game.pile)
        if error: continue
        #if draw: continue
       
        for j in range(2):
            stats_moves[j] += game.moves[j]
            stats_cheats[j] += game.cheats[j]
            stats_checks[j] += game.checks[j]
            stats_draw_decisions[j] += game.draw_decisions[j]
            stats_cards[j] += len(game.player_cards[j])

    div = repeats - errors
    if div > 0:
            
        stats_pile_size /= div          
        for j in range(2):
            stats_moves[j] /= div
            stats_cheats[j] /= div
            stats_checks[j] /= div
            stats_draw_decisions[j] /= div
            stats_cards[j] /= div
            
    return [stats_wins, stats_draws, stats_moves, stats_cards, stats_pile_size, stats_checks, 
            stats_draw_decisions, stats_cheats, stats_errors, errors, draws]  


# COMPARE

In [20]:
strategy = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["Rafalek", "Rafalek", Rafalek],
            ["Kot", "Kot", Kotterek3]
           ]


'strategy = [["RandomPlayer", "RANDOM", RandomPlayer],\n            ["Nazwisko", "Nazwisko", Nazwisko],\n            ["Kot1", "Kot1", MyFirstPlayer]\n           ]\n           '

In [21]:
#%pdb on

full_results = [[None for i in range(len(strategy))] for i in range(len(strategy))]

for A in range(len(strategy)):
    print("==== " + str(A), strategy[A][0])
    for B in range(A+1,len(strategy)):
        print(B, strategy[B][0])
        results = comparePlayers(strategy[A][2], strategy[B][2])
        full_results[A][B] = results
        


'\nfull_results = [[None for i in range(len(strategy))] for i in range(len(strategy))]\n\nfor A in range(len(strategy)):\n    print("==== " + str(A), strategy[A][0])\n    for B in range(A+1,len(strategy)):\n        print(B, strategy[B][0])\n        results = comparePlayers(strategy[A][2], strategy[B][2])\n        full_results[A][B] = results\n\n'

In [22]:
full_results

Simple stats

In [23]:
def printMatrix(full_results, stat):
    print(STAT_NAMES[stat])
    S = " "
    for s in strategy: S += (str(s[1]) + " " )
    print(S)
    for A in range(len(strategy)):
        print(A)
        S = str(strategy[A][1]) + " "
        for B in range(len(strategy)):
            if A == B: S += "- "
            elif A < B:
                S += str(full_results[A][B][stat][0]) + " "
            else:
                S += str(full_results[B][A][stat][1]) + " "
        print(S)
    
for a in ANALYZE_STATS:
    printMatrix(full_results, a)



'def printMatrix(full_results, stat):\n    print(STAT_NAMES[stat])\n    S = " "\n    for s in strategy: S += (str(s[1]) + " " )\n    print(S)\n    for A in range(len(strategy)):\n        print(A)\n        S = str(strategy[A][1]) + " "\n        for B in range(len(strategy)):\n            if A == B: S += "- "\n            elif A < B:\n                S += str(full_results[A][B][stat][0]) + " "\n            else:\n                S += str(full_results[B][A][stat][1]) + " "\n        print(S)\n\nfor a in ANALYZE_STATS:\n    printMatrix(full_results, a)\n'

# to już dodałem od siebie, tego nie ma w komparatorze!!!!
tu jest fragment kodu który zrobi ranking, na podstawie czysto liczby zwycięstw

In [24]:
from moje_dzieci.Kot_4 import Kot4
from moje_dzieci.Kot_5 import Kot5
from moje_dzieci.Kot_6 import Kot6

In [25]:
strategy = [
    # ["SecondPlayerOptim2", "SecondPlayerOptim2", MySecondPlayerOptim],
    ["Kot_First", "Kot_First", MyFirstPlayer],
    ["RandomPlayer", "RANDOM", RandomPlayer],
    # # # # # # # # ["Nazwisko", "Nazwisko", Nazwisko], # ten jest zaimportowany, ale jest pusty praktycznie, jak drawPlayer
    ["SimplePlayer", "SimplePlayer", SimplePlayer],
    ["NoCheck", "NoCheck", SimpleNoCheck],
    ["AlwaysCheck", "AlwaysCheck", SimpleAlwaysCheck],
    ["HonestPlayer", "HonestPlayer", HonestPlayer],
    # # # # # # # # ["DrawPlayer", "DrawPlayer", DrawPlayer], # no dobra, on jest zbyt chujowy by go brać pod uwagę ngl
    ["Kot_Second", "Kot_Second", MySecondPlayer],
    ["Kot_Third", "Kot_Third", MyThirdPlayer],
    ["FirstCardCounter", "FirstCardCounter", FirstCardCounter],
    ["SecondCardCounter", "SecondCardCounter", SecondCardCounter],
    ["Mr.Ciekawostka", "Mr.Ciekawostka", MySecondPlayer],
    # ["SecondPlayerOptim", "SecondPlayerOptim", MySecondPlayerOptim],
    ["Kondys_Dabrowski","Kondys_Dabrowski",Kondys_Dabrowski],
    ["Kot4","Kot4", Kot4],
    ["Kot5","Kot5", Kot5],
    ["Kot6","Kot6", Kot6]
   ]

#%pdb on
full_results = [[None for i in range(len(strategy))] for i in range(len(strategy))]

for A in range(len(strategy)):
    # print("==== " + str(A), strategy[A][0])
    for B in range(A+1,len(strategy)):
        # print(B, strategy[B][0])
        results = comparePlayers(strategy[A][2], strategy[B][2])
        full_results[A][B] = results


def generate_ranking(full_results, strategy):
    n = len(strategy)
    total_wins = [0 for _ in range(n)]

    for i in range(n):
        for j in range(n):
            if i < j and full_results[i][j] is not None:
                wins_i, wins_j = full_results[i][j][0]
                total_wins[i] += wins_i
                total_wins[j] += wins_j

    # Sparuj nazwy z wynikami
    ranking = list(zip([s[0] for s in strategy], total_wins))

    # Posortuj malejąco
    ranking.sort(key=lambda x: x[1], reverse=True)

    maxxx = (len(strategy) -1) * 1000

    print("RANKING GRACZY (według liczby wygranych):")
    for idx, (name, wins) in enumerate(ranking, 1):
        print(f"{idx}. {name} - {wins} / {maxxx} wygranych")

    return ranking

ranking = generate_ranking(full_results, strategy)

RANKING GRACZY (według liczby wygranych):
1. Kondys_Dabrowski - 11672 / 14000 wygranych
2. Kot4 - 10224 / 14000 wygranych
3. Kot_Second - 9871 / 14000 wygranych
4. Mr.Ciekawostka - 9842 / 14000 wygranych
5. Kot5 - 9494 / 14000 wygranych
6. Kot_Third - 9200 / 14000 wygranych
7. Kot_First - 8006 / 14000 wygranych
8. SimplePlayer - 7531 / 14000 wygranych
9. NoCheck - 7531 / 14000 wygranych
10. Kot6 - 6744 / 14000 wygranych
11. HonestPlayer - 6722 / 14000 wygranych
12. SecondCardCounter - 2462 / 14000 wygranych
13. AlwaysCheck - 1948 / 14000 wygranych
14. FirstCardCounter - 1941 / 14000 wygranych
15. RandomPlayer - 1812 / 14000 wygranych


In [26]:
import pandas as pd
import numpy as np

def create_win_matrix(full_results, strategy):
    names = [s[0] for s in strategy]
    n = len(names)
    matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i == j:
                matrix[i][j] = np.nan  # Brak pojedynków z samym sobą
            elif i < j and full_results[i][j] is not None:
                wins_i, wins_j = full_results[i][j][0]
                matrix[i][j] = wins_i
                matrix[j][i] = wins_j

    df = pd.DataFrame(matrix, index=names, columns=names)
    print("MACIERZ WYGRANYCH:")
    print(df.round(1))

    return df


df = create_win_matrix(full_results, strategy)

MACIERZ WYGRANYCH:
                   Kot_First  RandomPlayer  SimplePlayer  NoCheck  \
Kot_First                NaN         940.0         697.0    319.0   
RandomPlayer            60.0           NaN         102.0    613.0   
SimplePlayer           303.0         898.0           NaN    651.0   
NoCheck                681.0         387.0         349.0      NaN   
AlwaysCheck              0.0         987.0          10.0     75.0   
HonestPlayer           297.0         989.0         652.0    244.0   
Kot_Second             756.0         870.0         685.0    758.0   
Kot_Third              671.0         891.0         715.0    583.0   
FirstCardCounter         0.0         988.0           3.0     71.0   
SecondCardCounter        0.0         993.0          11.0     88.0   
Mr.Ciekawostka         781.0         887.0         669.0    726.0   
Kondys_Dabrowski       654.0        1000.0         932.0    585.0   
Kot4                   800.0         875.0         738.0    770.0   
Kot5           